## Project Topic - Understanding and Predicting Property Maintenance Fines and whether a blight ticket will                      be paid on time.

This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, our task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only a portion of data already collected is used for training the model for this project.** Nonetheless, I encourage everyone to look into data from other Detroit datasets to help inform feature creation and model selection.

___

We used two different data files in training and validating our models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

<br>

**File descriptions**

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

our predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

___

For this project, we create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    
    cols_to_use = ['ticket_id', 'violation_street_number', 'ticket_issued_date', 'hearing_date', 
               'violation_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 
               'judgment_amount', 'compliance']

    df = pd.read_csv('train.csv', encoding='latin1', usecols=cols_to_use, index_col='ticket_id')
    df.dropna(inplace=True)

    features_to_use = ['fine_amount', 'late_fee', 'discount_amount', 'judgment_amount']

    X_train = df[features_to_use]
    y_train = df['compliance']

    clf = GradientBoostingClassifier(random_state=0, n_estimators=100, learning_rate=0.1, max_depth=3)
    clf.fit(X_train, y_train)
    
    cols_to_use = ['ticket_id', 'fine_amount', 'late_fee', 'discount_amount', 'judgment_amount']
    X_test = pd.read_csv('test.csv', usecols=cols_to_use, index_col='ticket_id')

    y_predict_proba = clf.predict_proba(X_test)

    predict_dct = {}
    for item in zip(X_test.index, y_predict_proba):
        predict_dct.update({item[0]: item[1][1]})

    result_series = pd.Series(predict_dct)

    return result_series

blight_model()

284932    0.060052
285362    0.023170
285361    0.068376
285338    0.060052
285346    0.068376
            ...   
376496    0.023170
376497    0.023170
376499    0.068376
376500    0.068376
369851    0.261670
Length: 61001, dtype: float64

In [ ]:
## Note: I didn't write the evaluation/scoring code block as the model was evaluated automatically.

In [ ]:
## I got AUC score of 0.765282983033 with this model

In [2]:
# used for tuning the model
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV

# clf = GradientBoostingClassifier(random_state=0)
# params = {'learning_rate': [0.01, 0.1, 1, 10],
#           'n_estimators': [100, 200, 500],
#           'max_depth': [3, 5, 7, 10]}

# grid = GridSearchCV(clf, param_grid=params, scoring='roc_auc')
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_